In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer 
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel # QLoRA를 위한 준비 함수

# --- 모델 및 토크나이저 로드 ---
# Qwen1.5-4B 모델을 사용합니다. 정확한 모델 ID를 Hugging Face Hub에서 확인해 주세요.
# Qwen/Qwen1.5-4B-Chat 버전으로도 시도해 볼 수 있습니다.
model_id = "Qwen/Qwen3-4B" 

print(f"모델 '{model_id}'를 로드합니다...")
# 모델 로드 (GPU 메모리 절약을 위해 torch_dtype을 bfloat16으로 설정)
# Colab Pro 또는 고성능 GPU가 아니라면 4bit 양자화를 고려해 볼 수 있습니다.
# 만약 4bit 양자화를 사용하려면 BitsAndBytesConfig를 임포트하고 적용해야 합니다.
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
) 
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # BFloat16으로 로드 (GPU가 지원할 경우)
    device_map="auto",          # 사용 가능한 디바이스에 자동으로 매핑
    quantization_config=bnb_config # 4bit 양자화 사용 시 주석 해제
)

squad_finetuned_adapter_path = './qwen_squad_finetuned/squad_lora_adapter'
print(f"SQuAD 파인튜닝된 LoRA 어댑터 '{squad_finetuned_adapter_path}'를 로드하여 기본 모델에 연결합니다...")
# 기본 모델에 SQuAD 파인튜닝 어댑터를 연결합니다.
# 이제 'model' 객체는 SQuAD로 학습된 QA 능력을 가진 상태가 됩니다.
model = PeftModel.from_pretrained(base_model, squad_finetuned_adapter_path, trust_remote_code=True)
model.train() # 학습 모드로 설정 (SFTTrainer가 다시 설정해주지만 명시적으로)
print("SQuAD 어댑터 로드 및 연결 완료.")
# QLoRA 사용 시 모델을 K-bit 학습에 맞게 준비
model = prepare_model_for_kbit_training(model) # 4bit 양자화 사용 시 주석 해제

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 토크나이저 패딩 토큰 설정 (Qwen 모델은 일반적으로 pad_token이 명시적으로 설정되어 있지 않으므로 eos_token을 사용)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # 패딩 방향 설정: 주로 오른쪽 패딩이 선호됨

print("모델 및 토크나이저 로드 완료.")

모델 'Qwen/Qwen3-4B'를 로드합니다...


Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.50s/it]


SQuAD 파인튜닝된 LoRA 어댑터 './qwen_squad_finetuned/squad_lora_adapter'를 로드하여 기본 모델에 연결합니다...
SQuAD 어댑터 로드 및 연결 완료.
모델 및 토크나이저 로드 완료.


In [2]:
import json
from datasets import Dataset

# --- 데이터셋 로드 및 준비 ---
# 변환된 SQuAD 데이터셋 파일 경로 (2번 섹션에서 저장한 파일)
squad_qwen_format_path = 'korquad_qwen_format_processed.json'

print(f"데이터셋 '{squad_qwen_format_path}'를 로드합니다...")
with open(squad_qwen_format_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Hugging Face datasets 라이브러리의 Dataset 객체로 변환
dataset = Dataset.from_list(raw_data[:50000])
print(f"데이터셋 로드 완료. 총 {len(dataset)}개의 예제.")

# 데이터셋을 학습 세트와 검증 세트로 분할 (전체 데이터의 5%를 검증 세트로 사용)
# seed 값을 고정하여 매번 동일한 분할 결과 얻기
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']
print(f"학습 데이터셋 크기: {len(train_dataset)}, 검증 데이터셋 크기: {len(eval_dataset)}")

데이터셋 'korquad_qwen_format_processed.json'를 로드합니다...
데이터셋 로드 완료. 총 50000개의 예제.
학습 데이터셋 크기: 47500, 검증 데이터셋 크기: 2500


In [3]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [4]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)

transformers.training_args


In [5]:
from transformers import Trainer
print(Trainer.__module__)


transformers.trainer


In [ ]:

from peft import LoraConfig

# --- 파인튜닝 설정 (TrainingArguments) ---
output_dir = "./qwen_korquad_finetuned" # 학습된 모델이 저장될 경로

print("파인튜닝 설정을 정의합니다...")
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,      # GPU당 학습 배치 크기 (메모리 제약에 따라 조절)
    gradient_accumulation_steps=4,      # 그라디언트 누적 단계 (실제 배치 크기 = 2 * 4 = 8)
    learning_rate=2e-4,                 # 학습률
    num_train_epochs=3,                 # 학습 에포크 수
    logging_steps=100,                  # 몇 스텝마다 로그를 출력할지
    save_steps=500,                     # 몇 스텝마다 모델 체크포인트를 저장할지
    eval_strategy="steps",        # 스텝 단위로 검증 수행
    eval_steps=500,                     # 몇 스텝마다 검증을 수행할지
    save_total_limit=3,                 # 저장할 체크포인트 최대 개수
    fp16=True,                          # Float16 정밀도 학습 (GPU 지원 시 메모리 절약)
    report_to="none",                   # 학습 진행 상황 리포트 (wandb 등. 'none' 시 로컬 로그만 사용)
    load_best_model_at_end=True,        # 학습 종료 시 가장 성능이 좋았던 모델 로드
    metric_for_best_model="eval_loss",  # 최적 모델을 결정할 지표 (검증 손실이 가장 낮은 모델)
)

print("파인튜닝 설정 완료.")

# --- LoRA 설정 (PEFT) ---
print("LoRA 설정을 정의합니다...")
# r: LoRA의 랭크 (낮은 값은 더 작은 어댑터, 높은 값은 더 큰 어댑터)
# lora_alpha: LoRA 스케일링 팩터
# target_modules: LoRA를 적용할 모델의 모듈 이름. Qwen1.5 모델에 맞게 'q_proj', 'k_proj', 'v_proj', 'o_proj'를 일반적으로 타겟으로 합니다.
lora_config = LoraConfig(
    r=64, # LoRA 랭크 (일반적으로 8, 16, 32, 64)
    lora_alpha=16, # LoRA 스케일링 계수
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Qwen1.5의 주요 선형 레이어
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM", # 텍스트 생성 (인과적 언어 모델링)
)
print("LoRA 설정 완료.")

파인튜닝 설정을 정의합니다...
파인튜닝 설정 완료.
LoRA 설정을 정의합니다...
LoRA 설정 완료.


In [7]:
import numpy as np
from torch.serialization import add_safe_globals

# numpy._core.multiarray._reconstruct를 안전 목록에 추가
add_safe_globals([np._core.multiarray._reconstruct])



In [8]:
train_dataset[0]

{'messages': [{'content': '당신은 제공된 맥락에 따라 질문에 답하는 유용한 AI입니다. 반드시 한국어로 답 해주세요.',
   'role': 'system'},
  {'content': '맥락: 즉 이황의 설은 호발설이고 이이는 일도설이었다. 하지만 이황의 칠정설인 기발이승설만을 취한 것이다. 반면에 그의 사단설인 이발기수(理發氣隨)설을 비판하였다. 또한 그것은 이황과 사단칠정설논쟁을 벌인 기대승의 영향을 받은 것이다. 또한 서경덕에 대해서도 기중심의 설로서 독창적이지만 문제가 있다며 비판한다. 그의 설은 기가 운동하고 이는 그 원인이 된다는 설에 근거한 것이다. 그는 자기의 주장을 발전시키면서 이 주장이 주자의 뜻과 어긋나면 주자가 잘못 된 것이라고까지 하는 자신을 얻게 된 것이다. 이같이 그는 학문으로 유명할 뿐 아니라 경세가(經世家)로서도 혁혁한 업적을 남겼다. 훗날 영남의 유직이 효종원년에 올린 상소문에서 이이의 설을 불교와 육왕(육구연과 왕수인)과 같은 주기설로서 이단이라고 공격하였다. 그는 이황이야말로 주리설로서 정학이라는 사상에 근거하여 이이를 비판했던 것이다. 그후 주리설은 정학이고 주기설은 이단이라고 여겨지게 되었다. 그 상소로 인하여 유직은 조정으로부터 처벌받아 과거시험을 응시할 수 있는 자격을 박탈당하였다. 하지만 이이 역시 기의 뿌리가 이라고 말하였기 때문에 주리설이라고 할 수 있다. 또한 이황과 이이 모두 기의 뿌리가 리라고 했기 때문에 모두 이일원론 또는 이기일원론이라고 할 수 있다. 그의 저작인 《동호문답(東湖問答)》, 《성학집요(聖學輯要)》, 《인심도심설(人心道心說)》, 《시무육조소(時務六條疏)》 등은 모두 임금의 도리와 시무를 논한 명저로 그의 정치에 대한 태도는 유학자의 이상인 요순시대를 실현하는 것이었다.\n질문: 이이는 이황의 칠정성인 기발이승설은 취했는데. 반면에 비판한 사단설은 무엇인가?',
   'role': 'user'},
  {'content': '그것은 바로 이발기수(理發氣隨)설이랍니다.', 

In [ ]:
from peft import PeftModel, PeftConfig
import os
from trl import SFTTrainer, SFTConfig

# 기존 어댑터 로드
peft_config = PeftConfig.from_pretrained(r".\qwen_squad_finetuned\checkpoint-3000")
model = PeftModel.from_pretrained(model, r".\qwen_squad_finetuned\checkpoint-3000",is_trainable=True)

print("SFTTrainer를 초기화합니다...")
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, # 검증 데이터셋 추가
    peft_config=lora_config,
    #args=training_arguments,
    # False로 설정 시 각 예제가 독립적으로 처리됩니다.
    # SQuAD 맥락과 질문, 답변을 합친 길이가 이 값을 넘으면 잘려나갈 수 있습니다.
    args = SFTConfig(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4, 
        learning_rate=2e-4,
        num_train_epochs=3,
        logging_steps=10,
        save_steps=500,
        dataset_text_field="text",
        eval_strategy="steps", 
        eval_steps=500,    
        save_total_limit=3,
        disable_tqdm=False,
        fp16=True,
        report_to="none", 
        max_seq_length=512,
        load_best_model_at_end=True,
        packing=True,
        metric_for_best_model="eval_loss",
        resume_from_checkpoint=True
    )
    
)
print("SFTTrainer 초기화 완료. 학습을 시작합니다.")

# 모델 학습 시작


c:\Users\jiho1\anaconda3\envs\extra_llmenv\lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
c:\Users\jiho1\anaconda3\envs\extra_llmenv\lib\site-packages\peft\peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_mo

SFTTrainer를 초기화합니다...


Packing eval dataset: 100%|██████████| 2500/2500 [00:00<00:00, 356816.28 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


SFTTrainer 초기화 완료. 학습을 시작합니다.


In [10]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\jiho1\anaconda3\envs\extra_llmenv\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:

# 학습된 LoRA 어댑터 저장
output_adapter_dir = os.path.join(output_dir, "korquad_lora_adapter")
trainer.model.save_pretrained(output_adapter_dir)
tokenizer.save_pretrained(output_adapter_dir) # 토크나이저도 함께 저장

print(f"학습된 LoRA 어댑터가 '{output_adapter_dir}'에 저장되었습니다.")
print("이제 저장된 어댑터와 기본 모델을 병합하여 새로운 모델을 만들거나, 어댑터를 로드하여 추론에 사용할 수 있습니다.")

학습된 LoRA 어댑터가 './qwen_squad_finetuned\squad_lora_adapter'에 저장되었습니다.
이제 저장된 어댑터와 기본 모델을 병합하여 새로운 모델을 만들거나, 어댑터를 로드하여 추론에 사용할 수 있습니다.


In [ ]:
# 이 코드를 주피터 노트북의 새로운 코드 셀에 넣어 실행하세요.
# trainer.train()을 중단한 직후에 실행하면 됩니다.

import os
from datetime import datetime

# 저장할 디렉토리 경로 정의 (현재 시간으로 고유한 이름 생성)
# training_arguments.output_dir은 이전에 설정했던 'qwen_squad_finetuned'입니다.
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
manual_save_dir = os.path.join(training_arguments.output_dir, f"manual_save_{timestamp}")

print(f"현재까지 학습된 LoRA 어댑터를 '{manual_save_dir}'에 저장합니다...")

# 저장할 디렉토리가 없으면 생성
if not os.path.exists(manual_save_dir):
    os.makedirs(manual_save_dir)

# trainer 객체에서 현재 모델의 LoRA 어댑터 상태를 저장
# trainer.model은 PEFT (LoRA) 모델 객체입니다.
trainer.model.save_pretrained(manual_save_dir)

# 토크나이저도 함께 저장하여 나중에 이 어댑터를 로드할 때 사용할 수 있도록 합니다.
tokenizer.save_pretrained(manual_save_dir)

print(f"현재 학습 상태가 '{manual_save_dir}'에 성공적으로 저장되었습니다.")
print("이 어댑터를 로드하여 추론하거나, 나중에 학습을 재개할 때 사용할 수 있습니다.")

현재까지 학습된 LoRA 어댑터를 './qwen_squad_finetuned\manual_save_20250527-103754'에 저장합니다...
현재 학습 상태가 './qwen_squad_finetuned\manual_save_20250527-103754'에 성공적으로 저장되었습니다.
이 어댑터를 로드하여 추론하거나, 나중에 학습을 재개할 때 사용할 수 있습니다.


In [ ]:
# --- 학습된 모델로 추론하기 (선택 사항) ---
from peft import PeftModel
from transformers import pipeline

# 저장된 어댑터 로드
loaded_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    # quantization_config=bnb_config # 4bit 양자화 사용 시 주석 해제
)

# 어댑터를 기본 모델에 연결
adapter_path = os.path.join(output_dir, "korquad_lora_adapter")
model_with_adapter = PeftModel.from_pretrained(loaded_model, adapter_path)
print("어댑터 로드 및 연결 완료.")

# 추론을 위한 파이프라인 생성 (Qwen 토크나이저 사용)
# Qwen 모델은 채팅 템플릿 적용이 필요합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


def generate_answer(question_text, context_text, model, tokenizer):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
        {"role": "user", "content": f"맥락: {context_text}\n질문: {question_text}"}
    ]
    # 채팅 템플릿 적용
    # 이 부분은 Qwen1.5 모델의 최신 채팅 템플릿에 따라 약간 달라질 수 있습니다.
    # 일반적으로 apply_chat_template을 사용합니다.
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    # 입력 프롬프트 부분을 제외하고 생성된 텍스트만 디코딩
    generated_text = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return generated_text.strip()


# 예시 질문과 맥락
test_context = "Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child."
test_question = "When did Beyonce become famous?"

print("\n--- 테스트 추론 ---")
print(f"질문: {test_question}")
print(f"맥락: {test_context}")
answer = generate_answer(test_question, test_context, model_with_adapter, tokenizer)
print(f"답변: {answer}")


# --- LoRA 어댑터와 기본 모델 병합 및 저장 (선택 사항) ---
# 파인튜닝된 모델을 일반적인 형태로 저장하여 쉽게 배포하거나 사용할 수 있도록 합니다.
# GPU 메모리가 충분해야 이 작업을 수행할 수 있습니다.
# qlora를 사용했다면, merge_and_unload() 전에 prepare_model_for_kbit_training을 해제해야 할 수 있습니다.

print("\n--- LoRA 어댑터 병합 시작 (선택 사항) ---")
try:
    # QLoRA를 사용했다면 model_with_adapter.merge_and_unload()를 사용하기 전에
    # 모델을 CPU로 이동시키거나 (model_with_adapter.to('cpu'))
    # 일부 설정을 비활성화해야 할 수 있습니다.
    # 일반적인 LoRA에서는 바로 merge_and_unload()를 사용합니다.
    merged_model = model_with_adapter.merge_and_unload()
    print("어댑터 병합 완료.")

    # 병합된 모델 저장
    merged_model_output_dir = "./qwen_squad_finetuned_merged"
    merged_model.save_pretrained(merged_model_output_dir, safe_serialization=True)
    tokenizer.save_pretrained(merged_model_output_dir)
    print(f"병합된 모델이 '{merged_model_output_dir}'에 저장되었습니다.")

except Exception as e:
    print(f"모델 병합 중 오류 발생: {e}")
    print("GPU 메모리 부족이 원인일 수 있습니다. 병합은 많은 메모리를 요구합니다.")

print("작업 완료.")

Loading checkpoint shards: 100%|██████████| 3/3 [01:22<00:00, 27.56s/it]


어댑터 로드 및 연결 완료.

--- 테스트 추론 ---
질문: When did Beyonce become famous?
맥락: Beyoncé Giselle Knowles-Carter (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.
답변: <think>

</think>

late 1990s

--- LoRA 어댑터 병합 시작 (선택 사항) ---
어댑터 병합 완료.
병합된 모델이 './qwen_squad_finetuned_merged'에 저장되었습니다.
작업 완료.


In [ ]:
print("\n--- LoRA 어댑터 병합 시작 (선택 사항) ---")
try:
    # QLoRA를 사용했다면 model_with_adapter.merge_and_unload()를 사용하기 전에
    # 모델을 CPU로 이동시키거나 (model_with_adapter.to('cpu'))
    # 일부 설정을 비활성화해야 할 수 있습니다.
    # 일반적인 LoRA에서는 바로 merge_and_unload()를 사용합니다.
    merged_model = model_with_adapter.merge_and_unload()
    print("어댑터 병합 완료.")

    # 병합된 모델 저장
    merged_model_output_dir = "./qwen_squad_finetuned_merged"
    merged_model.save_pretrained(merged_model_output_dir, safe_serialization=True)
    tokenizer.save_pretrained(merged_model_output_dir)
    print(f"병합된 모델이 '{merged_model_output_dir}'에 저장되었습니다.")

except Exception as e:
    print(f"모델 병합 중 오류 발생: {e}")
    print("GPU 메모리 부족이 원인일 수 있습니다. 병합은 많은 메모리를 요구합니다.")

print("작업 완료.")